<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dist_sac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [0]:
from ape_x import *
import os
import multiprocessing as mp
import tensorflow as tf
import numpy as np
import time
import shutil

df = "audpred60.csv"

_ = shutil.copy("/content/drive/My Drive/"+df,"/content")
# sac_one_step.ckpt
saver_path = "sac_rsi2.ckpt"

restore = False

# if restore == True:
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".data-00000-of-00001","/content")
#   _ = shutil.copy("/content/drive/My Drive/" + saver_path + ".index","/content")
#   _ = shutil.copy("/content/drive/My Drive/checkpoint","/content")
  
if restore == True:
  _ = shutil.copy("/content/drive/My Drive/model/" + saver_path + ".data-00000-of-00001","/content")
  _ = shutil.copy("/content/drive/My Drive/model/"+ saver_path + ".index","/content")
  _ = shutil.copy("/content/drive/My Drive/model/checkpoint","/content")

output_size = 2
size = 240
num_actor = 3
mem_size = size * num_actor * 100
  
def actor_work(queues,epsilon, num):
  sess = tf.InteractiveSession()
  actor = Actor(df, 5, num, epsilon, sess,STEP_SIZE=size,OUTPUT_SIZE=output_size, save=False, saver_path=saver_path, restore=restore)
  actor.run(queues, 10, 1000,200,24,n=4)

def leaner_work(queues):
        sess = tf.InteractiveSession()
        leaner = Leaner(df, 5, sess,mem_size,output_size,'/gpu:0', save=True, saver_path=saver_path, restore=restore,noise=False, norm=False)
        leaner.leaner(queues)

In [0]:
%cd /content

queue = mp.Queue()
ps = [mp.Process(target=leaner_work, args=(queue,))]
for i in range(num_actor):
  epsilon = 0.0 if i == 0 else np.random.rand()
  ps.append(mp.Process(target=actor_work, args=(queue,epsilon,i)))
            
for p in ps:
    p.start()
    time.sleep(1)

for p in ps:
    p.join()

/content


/content/drive/My Drive/ape_x.py:483: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return gain.rolling(n).apply(rsiCalc)
/content/drive/My Drive/ape_x.py:199: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return gain.rolling(n).apply(rsiCalc)
/content/drive/My Drive/ape_x.py:199: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning

action probability =  [0.2217573221757322, 0.7782426778242678, 0.0]
trade accuracy =  0.47058823529411764
epoch: 10, total rewards: 3703.000000, mean rewards: 371.849372
action probability =  [0.42677824267782427, 0.5732217573221757, 0.0]
trade accuracy =  0.48739495798319327
epoch: 20, total rewards: 688.000000, mean rewards: 69.087866
action probability =  [0.8577405857740585, 0.14225941422594143, 0.0]
trade accuracy =  0.6092436974789915
epoch: 30, total rewards: 30996.000000, mean rewards: 3112.569038
action probability =  [0.2217573221757322, 0.7782426778242678, 0.0]
trade accuracy =  0.4495798319327731
epoch: 40, total rewards: -2258.000000, mean rewards: -226.744770
action probability =  [0.5439330543933054, 0.4560669456066946, 0.0]
trade accuracy =  0.46382978723404256
epoch: 50, total rewards: 801.000000, mean rewards: 80.435146
action probability =  [0.8577405857740585, 0.14225941422594143, 0.0]
trade accuracy =  0.5378151260504201
epoch: 60, total rewards: 12198.000000, mean

In [0]:
from multiprocessing import Process, Queue

def f(q):
  for i in range(100):
    rand = np.random.RandomState()
    if rand.rand() < 0.2:
      action = rand.choice(3, p=(0.2,0.3,0.5))
      print(i)
      print(action)

if __name__ == '__main__':
    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    p.join()

In [0]:
a = np.asanyarray([[1,1,1,1],[0,0,0,0]])

In [0]:
b = [i[0][0] for i in a]

In [0]:
import urllib.request
import urllib.parse


url = 'https://www.googleapis.com/drive/v2/files/trash'
f = urllib.request.urlopen(url)